In [86]:
import pandas as pd
import h5py
import numpy as np

filename = '/opt/data/o93o75/analysis/data_structures/fv_repA_3_18_2013_sal.mat'
condition = 'saline'
name = filename.split('/')[-1].split('.')[0]
f = h5py.File(filename,'r')
struarray  = f[name]

In [87]:
#extract values in hdf file into dictionary 
struct = {key:[] for key in struarray.keys()}
for k in struarray.keys():
    for n in range(struarray[k].shape[0]):
        if k == 'fn' or k == 'filename':
            struct[k].append(f[struarray[k][n,0]].value.tobytes()[::2].decode())#convert to string
        elif len(f[struarray[k][n,0]].shape)==1: #for nX0 dim vectors 
            struct[k].append([np.nan])
        else:
            struct[k].append(f[struarray[k][n,0]].value.flatten())
    numelem = [len(x) for x in struct[k]]
    if numelem[1:] == numelem[:-1] and max(numelem) == 1:#convert to single array if elements 
        #all have same size and = 1
        struct[k] = np.concatenate(struct[k]).ravel()

In [88]:
#create first dataframe for variables with indexing at trialnumber level
keys = ['datenm','firstpeakdistance','fn','ind','runlength','sm','smtmp']
df = pd.DataFrame(struct)
df['name'] = name
df['condition'] = condition
df['trialnumber'] = range(len(df))
df1 = df.set_index(['name','condition','trialnumber'])[keys]
df1.head()

datenm  firstpeakdistance  \
name                  condition trialnumber                                     
fv_repA_3_18_2013_sal saline    0            735311.422842                NaN   
                                1            735311.422855                NaN   
                                2            735311.422869                NaN   
                                3            735311.422884                NaN   
                                4            735311.422905                NaN   

                                                                        fn  \
name                  condition trialnumber                                  
fv_repA_3_18_2013_sal saline    0            o93o75_180313_1008.26296.cbin   
                                1            o93o75_180313_1008.26296.cbin   
                                2            o93o75_180313_1008.26296.cbin   
                                3            o93o75_180313_1008.26296.cbin   
                                4            o93o75_180313_1008.26296.cbin   

                                              ind  runlength  \
name                  condition trialnumber                    
fv_repA_3_18_2013_sal saline    0            25.0        3.0   
                                1            36.0        3.0   
                                2            47.0        3.0   
                                3            58.0        4.0   
                                4            69.0        3.0   

                                                                                            sm  \
name                  condition trialnumber                                                      
fv_repA_3_18_2013_sal saline    0            [0.0, 0.005701244933523253, 0.0095229791094335...   
                                1            [0.059615464590130396, 0.05962616044027027, 0....   
                                2            [0.05574348159545077, 0.05574752404617552, 0.0...   
                                3            [0.011061896324452244, 0.011157247040421455, 0...   
                                4            [0.0, 0.0038719084366301455, 0.007495017211501...   

                                                                                         smtmp  
name                  condition trialnumber                                                     
fv_repA_3_18_2013_sal saline    0            [-174.0, -174.0, -181.0, -185.0, -200.0, -190....  
                                1            [-157.0, -204.0, -199.0, -218.0, -206.0, -221....  
                                2            [-313.0, -305.0, -315.0, -305.0, -302.0, -292....  
                                3            [-225.0, -238.0, -226.0, -222.0, -242.0, -258....  
                                4            [-135.0, -156.0, -122.0, -166.0, -143.0, -118....

In [89]:
#create second dataframe for elements indexing at syllable level
struct2 = dict()
syllindex = np.concatenate(np.array([range(len(x)) for x in df.ons.values]))
trialnumindex = np.concatenate([np.repeat(df.trialnumber[x],len(df.ons[x])) for x in range(len(df))])
conditionindex = np.repeat(condition,len(trialnumindex))
nameindex = np.repeat(name,len(conditionindex))
struct2['syllgaps'] = np.concatenate([np.append(df.syllgaps[x],np.repeat(np.nan,len(df.ons[x])-
    len(df.syllgaps[x]))) for x in range(len(df.ons))])
struct2['sylldurations'] = np.concatenate(df.sylldurations)
struct2['ons'] = np.concatenate(df.ons)
struct2['off'] = np.concatenate(df.off)
index = pd.MultiIndex.from_arrays([nameindex,conditionindex,trialnumindex,syllindex],
                                  names=('name','condition','trialnumber','syllindex'))
df2 = pd.DataFrame(struct2,index=index)
df2[:20]

off       ons  \
name                  condition trialnumber syllindex                       
fv_repA_3_18_2013_sal saline    0           0          0.058625  0.013625   
                                            1          0.149281  0.091781   
                                            2          0.237313  0.182312   
                                1           0          0.061125  0.015500   
                                            1          0.157250  0.094750   
                                            2          0.244875  0.186438   
                                2           0          0.063625  0.016125   
                                            1          0.149156  0.093844   
                                            2          0.240563  0.185250   
                                3           0          0.066750  0.016125   
                                            1          0.153781  0.095969   
                                            2          0.247125  0.188375   
                                            3          0.341500  0.279625   
                                4           0          0.065500  0.015813   
                                            1          0.153000  0.095812   
                                            2          0.245000  0.187812   
                                5           0          0.067687  0.016438   
                                            1          0.154438  0.096937   
                                            2          0.249312  0.191500   
                                6           0          0.060812  0.015500   

                                                       sylldurations  syllgaps  
name                  condition trialnumber syllindex                           
fv_repA_3_18_2013_sal saline    0           0               0.045000  0.033156  
                                            1               0.057500  0.033031  
                                            2               0.055000       NaN  
                                1           0               0.045625  0.033625  
                                            1               0.062500  0.029188  
                                            2               0.058438       NaN  
                                2           0               0.047500  0.030219  
                                            1               0.055312  0.036094  
                                            2               0.055313       NaN  
                                3           0               0.050625  0.029219  
                                            1               0.057813  0.034594  
                                            2               0.058750  0.032500  
                                            3               0.061875       NaN  
                                4           0               0.049688  0.030312  
                                            1               0.057188  0.034812  
                                            2               0.057188       NaN  
                                5           0               0.051250  0.029250  
                                            1               0.057500  0.037062  
                                            2               0.057812       NaN  
                                6           0               0.045312  0.035969

In [90]:
f.close()